# Рекомендация тарифов

Имея данные о поведении клиентов, которые уже перешли на эти тарифы, нужно построить модель для задачи классификации, которая выберет подходящий тариф, с максимально большим значением *accuracy*. 

## Знакомство с данными

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 

from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression 

from sklearn.dummy import DummyClassifier

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [ ]:
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Пропусков нет. Типы данных правильные.

**Описание данных**

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
* сalls — количество звонков,
* minutes — суммарная длительность звонков в минутах,
* messages — количество sms-сообщений,
* mb_used — израсходованный интернет-трафик в Мб,
* is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Разбиение данных на выборки

Разделим исходные данные на обучающую, валидационную и тестовую выборки, в отношении 3 : 1 : 1

In [ ]:
features = df.drop(['is_ultra'], axis=1) # извлечем признаки
target = df['is_ultra'] # извлечем целевой признак

In [ ]:
features.shape

(3214, 4)

In [ ]:
target.shape

(3214,)

In [ ]:
# отделим 20% данных для тестовой выборки
features_train_valid, features_test, target_train_valid, target_test = train_test_split(
    features, target, test_size=0.2, random_state=12345) 

In [ ]:
features_test.shape # признаки, тестовая выборка

(643, 4)

In [ ]:
target_test.shape # целевой признак, тестовая выборка

(643,)

In [ ]:
# отделим 25% данных для валидационной выборки
features_train, features_valid, target_train, target_valid = train_test_split(
    features_train_valid, target_train_valid, test_size=0.25, random_state=12345) 

In [ ]:
features_train.shape # признаки, тренировочная выборка

(1928, 4)

In [ ]:
target_train.shape # целевой признак, тренировочная выборка

(1928,)

In [ ]:
features_valid.shape # признаки, валидационная выборка

(643, 4)

In [ ]:
target_valid.shape # целевой признак, валидационная выборка

(643,)

<div class="alert alert-block alert-success">
<b>Успех:</b> Разбиение было сделано правильно. Отлично, что размеры полученных наборов были напечатаны!
</div>

## Исследование моделей

3.1 Дерево решений

In [ ]:
best_model_DT = None
best_accuracy = 0
best_depth = 0

for depth in range(1, 11): # в цикле меняем гиперпараметр - максимальную глубину дерева
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) # модель - дерево решений, с заданной глубиной дерева
    model.fit(features_train, target_train) # алгоритм обучения - на выходе обученная модель на тренировочной выборке
    predictions = model.predict(features_valid) # предсказания на валидационной выборке
    accuracy = accuracy_score(target_valid, predictions) # доля правильных ответов - качество модели
    if accuracy > best_accuracy:
        best_model_DT = model
        best_accuracy = accuracy
        best_depth = depth
        
print("Accuracy лучшей модели 'Дерево решений' на валидационной выборке:", best_accuracy, "Глубина дерева:", best_depth)

Accuracy лучшей модели 'Дерево решений' на валидационной выборке: 0.7744945567651633 Глубина дерева: 7


3.2 Случайный лес

In [ ]:
best_model_RF = None
best_accuracy = 0
best_est = 0

for est in range(1, 51): # в цикле меняем гиперпараметр - максимальное количество деревьев
    model = RandomForestClassifier(random_state=12345, n_estimators=est) # модель - случайный лес, с заданным количеством деревьев
    model.fit(features_train, target_train) # алгоритм обучения - на выходе обученная модель на тренировочной выборке
    accuracy = model.score(features_valid, target_valid) # доля правильных ответов - качество модели на валидационной выборке
    if accuracy > best_accuracy:
        best_model_RF = model
        best_accuracy = accuracy
        best_est = est
        
print("Accuracy лучшей модели 'Случайный лес' на валидационной выборке:", best_accuracy, "Количество деревьев:", best_est)

Accuracy лучшей модели 'Случайный лес' на валидационной выборке: 0.7947122861586314 Количество деревьев: 44


3.3 Логистическая регрессия

In [ ]:
model = LogisticRegression(random_state=12345) # модель логистической регрессии с параметром random_state=12345
model.fit(features_train, target_train) # алгоритм обучения - на выходе обученная модель на тренировочной выборке
accuracy = model.score(features_valid, target_valid) # доля правильных ответов - качество модели на валидационной выборке

print("Accuracy модели логистической регрессии на валидационной выборке:", accuracy)

Accuracy модели логистической регрессии на валидационной выборке: 0.6967340590979783


На валидационной выборке лучшее качество показала модель "Случайный лес": 0.7947122861586314, Количество деревьев: 44 

## Проверка модель на тестовой выборке

In [ ]:
predictions = best_model_DT.predict(features_test) # предсказания на тестовой выборке
# доля правильных ответов - качество модели 'Дерево решений' на тестовой выборке
accuracy = accuracy_score(target_test, predictions) 
print("Accuracy лучшей модели 'Дерево решений' на тестовой выборке:", accuracy, "Глубина дерева:", best_depth)

Accuracy лучшей модели 'Дерево решений' на тестовой выборке: 0.7884914463452566 Глубина дерева: 7


In [ ]:
# доля правильных ответов - качество модели "Случайный лес" на тестовой выборке
accuracy = best_model_RF.score(features_test, target_test) 
print("Accuracy лучшей модели 'Случайный лес' на тестовой выборке:", accuracy, "Количество деревьев:", best_est)

Accuracy лучшей модели 'Случайный лес' на тестовой выборке: 0.7916018662519441 Количество деревьев: 44


In [ ]:
# доля правильных ответов - качество модели логистической регрессии на тестовой выборке
accuracy = model.score(features_test, target_test) 
print("Accuracy модели логистической регрессии на тестовой выборке:", accuracy)

Accuracy модели логистической регрессии на тестовой выборке: 0.702954898911353


На тестовой выборке лучшая модель "Случайный лес" (количество деревьев: 44) показала качество чуть хуже, чем на валидационной выборке: 0.7916018662519441. Будем считать, что лучшая модель достаточно хорошо обучена. Не переобучена и не недообучена.

## Проверка модели на адекватность

Чтобы оценить адекватность модели в задачах классификации, нужно сравнить её со случайной. 

In [ ]:
dummy_clf = DummyClassifier(strategy="most_frequent", random_state=12345)
#dummy_clf = DummyClassifier(strategy="stratified", random_state=12345)
#dummy_clf = DummyClassifier(strategy="prior", random_state=12345)
#dummy_clf = DummyClassifier(strategy="uniform", random_state=12345)
dummy_clf.fit(features_train, target_train) # алгоритм обучения - на выходе обученная модель на тренировочной выборке

DummyClassifier(constant=None, random_state=12345, strategy='most_frequent')

In [ ]:
# доля правильных ответов - качество случайной модели на валидационной выборке
accuracy = dummy_clf.score(features_valid, target_valid) 
print("Accuracy случайной модели на валидационной выборке:", accuracy)

Accuracy случайной модели на валидационной выборке: 0.6889580093312597


In [ ]:
# доля правильных ответов - качество случайной модели на тестовой выборке
accuracy = dummy_clf.score(features_test, target_test) 
print("Accuracy случайной модели на тестовой выборке:", accuracy)

Accuracy случайной модели на тестовой выборке: 0.6951788491446346


Качество случайной модели на валидационной и на тестовой выборках хуже, чем показала лучшая модель "Случайный лес". Будем считать, что лучшая модель адекватна.  

<div class="alert alert-block alert-warning">
<b>Выводы:</b> 
   
На предложенной в проекте выборке было исследовано три вида моделей для задачи классификации (с изменением гиперпараметров):
    
* дерево решений
* случайный лес
* логистическая регрессия
    
Лучшее качество на валидационной выборке показала модель "Случайный лес" (количество деревьев: 44): 0.7947122861586314.   
    
На тестовой выборке лучшая модель "Случайный лес" (количество деревьев: 44) показала качество чуть хуже, чем на валидационной выборке: 0.7916018662519441. Будем считать, что лучшая модель достаточно хорошо обучена. Не переобучена и не недообучена.
    
Качество случайной модели на валидационной и на тестовой выборках хуже, чем показала лучшая модель "Случайный лес". Будем считать, что лучшая модель адекватна.    
</div>